#### Human Protein Atlas Image Classification

URL: https://www.kaggle.com/c/human-protein-atlas-image-classification

In [35]:
import os
import skimage
import pandas as pd
import numpy as np
from skimage.transform import resize
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer

In [36]:
def get_x_y(type_dataset):
    multilabel = MultiLabelBinarizer()
    x_names = []
    x = []
    y = []
    df = pd.read_csv('datasets/' + type_dataset + '.csv', index_col=0)
    for indice_fila in df.iterrows():
        x_names.append(indice_fila[0])
        labels = (indice_fila[1]['Target']).split(" ")
        labels = [int(i) for i in labels]
        y.append(labels)
    
    y_multilabel = multilabel.fit_transform(y)
    for names in x_names:
        path = './datasets/' + type_dataset + '/' + names
    
        img_blue = np.array(Image.open(path + '_blue.png'))
        img_blue = np.asarray(skimage.transform.resize(img, (150, 150, 3)))
        
        img_green = np.array(Image.open(path + '_green.png'))
        img_green = np.asarray(skimage.transform.resize(img, (150, 150, 3)))
        
        img_red = np.array(Image.open(path +'_red.png'))
        img_red = np.asarray(skimage.transform.resize(img, (150, 150, 3)))
        
        img_yellow = np.array(Image.open(path + '_yellow.png'))
        img_yellow = np.asarray(skimage.transform.resize(img, (150, 150, 3)))
    
        result = np.concatenate((img_blue, img_green, img_red, img_yellow), axis=2)
        x.append(result)

    x = np.array(x)
    return x, y_multilabel


In [37]:
x_train, y_train = get_x_y('train')
print(type(y_train))
print(type(x_train))
print(y_train.shape[1])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
28


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [30]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers

In [33]:
dropout = 0.25
output_size = y_multilabel.shape[1]

model = Sequential()
model.add(Convolution2D(filters=6, strides=1, kernel_size=6, padding='same',input_shape=(150,150,12)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Convolution2D(filters=12, strides=2, kernel_size=5, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Convolution2D(filters=24, strides=2, kernel_size=4, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Flatten())
model.add(Dense(200))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))
model.add(Dropout(dropout))
model.add(Dense(output_size, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 150, 150, 6)       2598      
_________________________________________________________________
batch_normalization_5 (Batch (None, 150, 150, 6)       24        
_________________________________________________________________
activation_5 (Activation)    (None, 150, 150, 6)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 150, 150, 6)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 75, 75, 12)        1812      
_________________________________________________________________
batch_normalization_6 (Batch (None, 75, 75, 12)        48        
_________________________________________________________________
activation_6 (Activation)    (None, 75, 75, 12)        0         
__________

In [ ]:
lr = 0.001
plot_interval = 1
epochs = 80
batch_size = 512

adam = optimizers.Adam(lr=lr)
model.compile(loss = 'binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(filepath='Test1.weights.best.hdf5', verbose=1, save_best_only=True)
plot_losses = PlotLosses(plot_interval=plot_interval, evaluate_interval=None, x_val=x_test, y_val_categorical=y_test)

model.fit(x_train, 
          y_train,
          epochs=epochs, 
          batch_size=batch_size, 
          #verbose=1, 
          validation_data=(x_test, y_test), 
          callbacks=[plot_losses, 
                     checkpointer],
         )

In [ ]:
model.load_weights('Test1.weights.best.hdf5')
score = model.evaluate(x_val, y_val, verbose=0)
print("loss: ", score[0])
print("accuracy: ", score[1])

In [ ]:
model.load_weights('Test1.weights.best.hdf5')
plot_confusion_matrix(model.predict_classes(x_val), y_val)